In [ ]:
# -*- coding:utf-8 -*-
import cv2
import facenet

In [ ]:
import dlib

# load camera

In [ ]:
cap = cv2.VideoCapture(0)
cascade_path = "./haarcascade_frontalface_default.xml"

In [ ]:
face_detector = dlib.get_frontal_face_detector()


# face detect

In [ ]:
while True:
    _, frame = cap.read()
    
    if frame is None:
        print("lost camera")
        continue
#     else:
#         frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    detected_faces = face_detector(frame, 1)
    facerect = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]
    if len(facerect) > 0:
        print('face detected')
#         color = (255, 255, 255)  # 白
        color = (0,255,0) # 綠
        for rect in facerect:
            # 找出人臉位置的地方, 並用長方形框起來
        
            cv2.rectangle(frame, tuple(rect[0:2]), tuple(rect[2:4]), color, 2)
#             cv2.rectangle(frame,(384,0),(510,128),(0,255,0),3)

            x, y = rect[0:2]
            width, height = rect[2:4]
            image = frame[y - 10: y + height, x: x + width]
            cv2.putText(frame,"Hello World!!!", (x,y + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, fontScale  = 1,
                        color = color,
                        thickness=2)

            #                 result = model.predict(image)
#                 if result == 0:  # boss
#                     print('Boss is approaching')

#                 else:
#                     print('Not boss')
    cv2.imshow("face dection", frame)
                
    #10msecキー入力待ち
    k = cv2.waitKey(100)
    #Escキーを押されたら終了
    if k == 27:
        break

    #キャプチャを終了
cap.release()
cv2.destroyAllWindows()

In [ ]:
failed_image_list = []
with tf.Graph().as_default():
      
        with tf.Session() as sess:
            
            np.random.seed(seed=666)
            
            
            dataset = facenet.get_dataset(data_dir)

            # Check that there are at least one training image per class
            for cls in dataset:
                assert(len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset')            

                 
            paths, labels = facenet.get_image_paths_and_labels(dataset)
            print('Number of classes: %d' % len(dataset))
            print('Number of images: %d' % len(paths))
            
            # Load the model
            print('Loading feature extraction model')
            facenet.load_model(model)
            
            # Get input and output tensors
            images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
            embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
            phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
            embedding_size = embeddings.get_shape()[1]
            
            # Run forward pass to calculate embeddings
            print('Calculating features for images')
            nrof_images = len(paths)
            nrof_batches_per_epoch = int(math.ceil(1.0*nrof_images / batch_size))
            emb_array = np.zeros((nrof_images, embedding_size))
            for i in range(nrof_batches_per_epoch):
                start_index = i*batch_size
                end_index = min((i+1)*batch_size, nrof_images)
                paths_batch = paths[start_index:end_index]
                images = facenet.load_data(paths_batch, False, False, image_size)
                feed_dict = { images_placeholder:images, phase_train_placeholder:False }
                emb_array[start_index:end_index,:] = sess.run(embeddings, feed_dict=feed_dict)
            
            classifier_filename_exp = os.path.expanduser(classifier_filename)

            if (mode=='TRAIN'):
                # Train classifier
                print('Training classifier')
                model = SVC(kernel='linear', probability=True)
                model.fit(emb_array, labels)
            
                # Create a list of class names
                class_names = [ cls.name.replace('_', ' ') for cls in dataset]

                # Saving classifier model
                with open(classifier_filename_exp, 'wb') as outfile:
                    pickle.dump((model, class_names), outfile)
                print('Saved classifier model to file "%s"' % classifier_filename_exp)
                
            elif (mode=='CLASSIFY'):
                # Classify images
                print('Testing classifier')
                with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)

                print('Loaded classifier model from file "%s"' % classifier_filename_exp)

                predictions = model.predict_proba(emb_array)
                best_class_indices = np.argmax(predictions, axis=1)
                best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]

                for i in range(len(best_class_indices)):
                    image_name = paths[i].split("/")[-1]
                    person_name = image_name.split("\\")[0]
#                     print('ground truth name %s: my predict = %s' % (, class_names[best_class_indices[i]]))
#                     print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
                    if(person_name != class_names[best_class_indices[i]]):
                        print(image_name)
                        failed_image_list.append(paths[i])
                        
                accuracy = np.mean(np.equal(best_class_indices, labels))
                print('Accuracy: %.3f' % accuracy)